In [ ]:
# uncomment the following if you had not installed the package
# import sys
# sys.path.append('/home/khan/Documents/developments/pyIntertidalDEM')

from pyintdem.data import Database, DataFile
from pyintdem.workflows.khan2019 import create_mask
from pathlib import Path
import rioxarray
from tqdm import tqdm

# Example for downloaded dataset

In [ ]:
datadir = Path('/run/media/khan/Workbench/Developments/Bengal/Data')
database = Database(datadir)

In [ ]:
database

In [ ]:
maskdir = Path('/run/media/khan/Workbench/Developments/Bengal/Masks')
create_mask(database=database, maskdir=maskdir, nmask=0.5, ext='tif', band='B11')

In [ ]:
database.tiles

# Example for Pekel et al. dataset

In [ ]:
rootdir = Path('F:/Sentinel2')
datadir = rootdir / 'Data'
database = Database(datadir)

In [ ]:
# Merging Pekel dataset
# Run once only
from rioxarray.merge import merge_arrays

pekel_dir = Path('F:/Sentinel2/Auxiliary/Pekel_etal_2016/')

fnames = [
    'occurrence_40E_10Sv1_4_2021.tif',
    'occurrence_40E_20Sv1_4_2021.tif',
    'occurrence_50E_10Sv1_4_2021.tif',
    'occurrence_50E_20Sv1_4_2021.tif'
]

datasets = [rio.open_rasterio(pekel_dir / fname) for fname in fnames]
merged_ds = merge_arrays(datasets)
merged_ds.rio.to_raster('F:/Sentinel2/Auxiliary/Pekel_etal_2016/occurrence_madagascar.tif')

In [ ]:
ds_mask_global = rioxarray.open_rasterio('F:/Sentinel2/Auxiliary/Pekel_etal_2016/occurrence_madagascar.tif')

In [ ]:
maskdir = rootdir / 'Masks_Pekel'
if not maskdir.exists():
    maskdir.mkdir()

for tile in tqdm(database):
    ds_band = rioxarray.open_rasterio(database[tile][0]['bands']['B11'])
    ds_mask_local = ds_mask_global.rio.reproject_match(ds_band)

    # Mask land 0, water 1
    ds_mask_local = (ds_mask_local >= 25).astype(int)
    ds_mask_local.rio.to_raster(maskdir / f'{tile}.tif')

In [ ]:
database_test = {key: database[key] for key in  ['T39KTV', 'T38KMG', 'T39KTS']}
maskdir = rootdir / 'Masks_Sentinel2_Updated'
if not maskdir.exists():
    maskdir.mkdir()
create_mask(database=database_test, maskdir=maskdir, nmask=0.75, ext='tif', band='B11', normalize=True)